In [2]:
from read_db import read_db
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

weather, running, full, stat= read_db()



In [4]:
datasets=stat[['count','dayofweek','mean_tempf','max_tempf','precipitationIn',
	'mean_dew_pointf','mean_hum','mean_wind_speed','wind_dir_degrees',
	'mean_visibility_miles','cloudcover','mean_sea_level_pressure_in','events']]
 
datasets['events']=datasets['events'].fillna(value = 'Clear')

datasets['id'] = datasets['events'].astype('category').cat.codes

training=datasets[['dayofweek','id','mean_tempf','max_tempf','precipitationIn',
	'mean_dew_pointf','mean_hum','mean_wind_speed','wind_dir_degrees',
	'mean_visibility_miles','cloudcover','mean_sea_level_pressure_in','count']]
training['precipitationIn']=training['precipitationIn'].replace('T',0.0001)
training['precipitationIn']=training['precipitationIn'].astype(float)

/home/yanxin/anaconda2/envs/insight/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yanxin/anaconda2/envs/insight/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yanxin/anaconda2/envs/insight/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [6]:
%matplotlib inline
##GLM modeling, plot with stat parameters
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_predict
from sklearn import linear_model
import numpy as np

def computing_stat(X, Y):
   lr = LinearRegression()
   lr.fit(X, Y)
#     r^2 of model
#     lr.score(X, Y)
#     # The mean square error
#     print("Residual sum of squares: %.2f"
#           % np.mean((lr.predict(X) - Y) ** 2))
#     # Explained variance score: 1 is perfect prediction
#     print('Variance score: %.2f' % lr.score(X, Y))
   return np.mean((lr.predict(X) - Y) ** 2), lr.score(X, Y)

#return least sig feature
def remove_one_feature(X, Y, names):
   lr = LinearRegression()
   rfe = RFE(lr, n_features_to_select=1)
   rfe.fit(X,Y)
   rank = (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names)))
   return rank[-1][1]

total_feature = 12
drop_feature = [] #str type
num_feature = []
r2 = []
rss = []
feature = []
for i in range(total_feature-5):
   array = training.values.astype(int)
   Y = array[:,-1]
   X = array[:, 0:-2]
   names = training.columns
   rss_temp, r2_temp = computing_stat(X, Y)
#     print("fished stats")
   rss.append(rss_temp)
   r2.append(r2_temp)
   num_feature.append(total_feature-i)
   drop = remove_one_feature(X, Y, names)
#     print("fished remove one "+i)
   drop_feature.append(drop)
   training = training.drop(drop, axis=1)
   #print(i)

In [10]:
rss

[965.6801963061406,
 901.41536975719646,
 902.84750397925336,
 902.85482731643447,
 903.34182474041347,
 903.72809214998415,
 908.44533169662236]

In [11]:
r2

[0.0011684271223409981,
 0.067639436912255269,
 0.066158138152002444,
 0.066150563407875618,
 0.065646847576443013,
 0.065247319776526758,
 0.060368139470218025]

In [12]:
drop_feature

['mean_tempf',
 'wind_dir_degrees',
 'mean_wind_speed',
 'id',
 'cloudcover',
 'mean_hum',
 'mean_dew_pointf']